In [ ]:
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

In [ ]:
!pip install langchain --upgrade
!pip install spotipy
!pip install replicate
!pip install python-dotenv

In [ ]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

In [ ]:
with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

In [ ]:
import spotipy.util as util
from langchain.requests import RequestsWrapper
from decouple import config

from dotenv import load_dotenv
load_dotenv()

def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(raw_spec['components']['securitySchemes']['oauth_2_0']['flows']['authorizationCode']['scopes'].keys())
    access_token = util.prompt_for_user_token(scope=','.join(scopes))
    return {
        'Authorization': f'Bearer {access_token}'
    }

headers = construct_spotify_auth_headers(raw_spotify_api_spec)
print(headers)
requests_wrapper = RequestsWrapper(headers=headers)

In [ ]:
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, AgentOutputParser
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import SerpAPIWrapper
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
from langchain.output_parsers import RetryWithErrorOutputParser
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

llm = OpenAI(model_name="gpt-4", temperature=0.0)

class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            llm = OpenAI(temperature=0.0)
            return self.parse(llm(f"""Format the following text according to the format instructions given below:
            
{llm_output}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [SpotifyAPI, Google Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
"""))
            # raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
        
parser = CustomOutputParser()

spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm=llm)

search = SerpAPIWrapper()

tools = [
    Tool(
        name="SpotifyAPI",
        func=spotify_agent,
        description="Useful for getting information from the spotify api. pass the entire task to be completed in spotify, as this tool can support multiple steps"
    ),
    Tool(
        name="Google Search",
        func=search.run,
        description="Useful for searching Google for songs that fit a particular prompt, musical history, musical facts, and more"
    )
]

prefix = """You are a music assistant that helps create playlists. You have access to the following tools:"""
suffix = """Begin!

You should pass the entire spotify-related request to SpotifyAPI.

Question: {input}
{agent_scratchpad}"""


prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names, output_parser=parser)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

user_input = input("Please enter your message: ")
agent_executor.run(user_input)